<a href="https://colab.research.google.com/github/emmab-collab/PyTorch/blob/main/PyTorch_ResNet_Digestive_Biopsy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=5

In [2]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader

from medmnist import INFO, PathMNIST

# **1. Preparing the data**

In [3]:
from torchvision.transforms import ToTensor, Normalize, Compose

In [4]:
transform = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))  # normalise en centrant autour de 0, en divisant par 0.5 (par exemple)
])

train_dataset = PathMNIST(split='train', download=True, transform=transform)
val_dataset = PathMNIST(split='val', download=True, transform=transform)
test_dataset = PathMNIST(split='test', download=True, transform=transform)

100%|██████████| 206M/206M [03:38<00:00, 941kB/s] 


In [ ]:
len(train_dataset)

In [ ]:
repartition=torch.bincount(torch.tensor(train_dataset.labels.squeeze())).tolist()

In [ ]:
repartition/np.sum(repartition)

In [ ]:
train_dataset

In [ ]:
img,label=train_dataset[0]
print(img.shape,label)
img

In [ ]:
batch_size=100


train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
data_flag = 'pathmnist'
info = INFO[data_flag]
labels = info['label']
print(f"Les labels: \n{labels}\n")

In [ ]:
labels['0']

In [ ]:
num_classes = len(labels)
print(f"Nombre de classes : {num_classes}")

# **2. Visualising images**

In [ ]:
import matplotlib.pyplot as plt
#on change les tensor to 32*32*3
#parce que dans matplotlib la dim de la couleur est à droite

def show_example(img,label):
    print(f'Label: {labels[str(label[0])]}')
    plt.imshow(img.permute(1,2,0))

In [ ]:
show_example(*train_dataset[0])

In [ ]:
show_example(*train_dataset[1099])

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images,labels in dl:
        fig,ax=plt.subplots(figsize=(10,10))
        ax.set_xticks([]);ax.set_yticks([])
        ax.imshow(make_grid(images,10).permute(1,2,0))
        break

In [ ]:
show_batch(train_dl)

# **3. Defining the Model (WideResNet22)**

**22** : 22 convolutional layers

**residual blocs** : adds the original input ack to the output feature map obtained by passing the input through one or more convolutional layers.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1)
        self.relu1=nn.ReLU()
        self.conv2=nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()

    def forward(self,x):
        out=self.conv1(x)
        out=self.relu1(out)
        out=self.conv2(out)
        return self.relu2(out + x)

In [ ]:
simple_resnet=SimpleResidualBlock()

for images,labels in train_dl:
    out=simple_resnet(images)
    print(out.shape)
    break

In [ ]:
def conv_2d(ni,nf,stride=1,ks=3):
    return nn.Conv2d(in_channels=ni,out_channels=nf,
                    kernel_size=ks,stride=stride,
                    padding=ks//2,bias=False)

def bn_relu_conv(ni,nf):
    return nn.Sequential(nn.BatchNorm2d(ni),
                       nn.ReLU(inplace=True),
                       conv_2d(ni,nf))

class ResidualBlock(nn.Module):
    def __init__(self,ni,nf,stride=1):
        super().__init__()
        self.bn=nn.BatchNorm2d(ni)
        self.conv1=conv_2d(ni,nf,stride)
        self.conv2=bn_relu_conv(nf,nf)
        self.shortcut=lambda x: x #c'est une fonction qui retourne l'input
        if ni!=nf:
            self.shortcut=conv_2d(ni,nf,stride,1)

    def forward(self,x):
        x=F.relu(self.bn(x),inplace=True)
        r=self.shortcut(x)
        x=self.conv1(x)
        x=self.conv2(x)*0.2 #scaling factor (ca marche mieux)
        return x.add_(r)

In [ ]:
def make_group(N,ni,nf,stride):
    start=ResidualBlock(ni,nf,stride)
    rest=[ResidualBlock(nf,nf)for j in range(1,N)]
    return [start]+rest

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x.reshape(x.size(0),-1)

class WideResNet(nn.Module):
    def __init__(self,n_groups,N,n_classes,k=1,n_start=16):
        super().__init__()
        #increase channels to n_start using conv layer
        layers=[conv_2d(3,n_start)]
        n_channels=[n_start]

        #add groups of BasicBlock(increase channels and downsample)
        for i in range(n_groups):
            n_channels.append(n_start*(2**i)*k)
            stride=2 if i>0 else 1
            layers += make_group(N,n_channels[i],
                                n_channels[i+1],stride)

        #Pool, flatten and and linear layer for classification
        layers += [nn.BatchNorm2d(n_channels[3]),
                  nn.ReLU(inplace=True),
                  nn.AdaptiveAvgPool2d(1),
                  Flatten(),
                  nn.Linear(n_channels[3],n_classes)]

        self.features=nn.Sequential(*layers)

    def forward(self,x):
        return self.features(x)

def wrn_22():
    return WideResNet(n_groups=3,N=3,n_classes=10,k=6)

In [ ]:
model=wrn_22()

In [ ]:
model

In [ ]:
for images,labels in train_dl:
    print(f'images.shape : {images.shape}')
    out=model(images)
    print(f'out.shape : {out.shape}')
    break

# **4. Training the model**

``data = DataLoaders.from_dsets(train_ds, val_ds, bs=batch_size)`` : on crée un dataloader à partir de deux ensembles de données PyTorch (train et validation)

``learner = Learner(data, model, loss_func=F.cross_entropy, metrics=[accuracy])`` : on crée un learner, objet central dans fast.ai
 - Ce Learner va s’occuper de tout : entraînement, évaluation, sauvegarde, affichage de résultats, etc.

``learner.add_cb(GradientClip(0.1))`` : on ajoute une "callback" qui va empêcher les gradients d’exploser
 - ``GradientClip(0.1)`` : empêche les gradients d’avoir une norme plus grande que 0.1 pendant la backpropagation
 - C’est utile si ton modèle est instable à l’entraînement (perte qui diverge)

In [ ]:
from fastai.vision.all import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data = DataLoaders.from_dsets(train_ds, val_ds, bs=batch_size,device=device)

learner = Learner(
    data,
    model,
    loss_func=F.cross_entropy,
    metrics=[accuracy]
)

learner.add_cb(GradientClip(0.1))

In [ ]:
print(type(learner))

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(9,5e-3,wd=1e-4)

In [ ]:
learner.recorder.plot_loss()